In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
df = pd.read_csv('data_aa2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 12 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   Unnamed: 0     int64 
 1   user_id        int64 
 2   order_id       int64 
 3   product_id     int64 
 4   product_name   object
 5   aisle_id       int64 
 6   aisle          object
 7   department_id  int64 
 8   department     object
 9   order_number   int64 
 10  reordered      int64 
 11  is_organic     bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 2.7+ GB


In [21]:


# 사용자별 주문당 제품 개수 계산
user_product_counts = df.groupby(['user_id', 'order_id']).size().reset_index(name='product_count')

# 각 사용자별 최대 제품 개수만 남기기
user_max_product_count = user_product_counts.groupby('user_id')['product_count'].max().reset_index()

# 최대 제품 개수 기준으로 사업자와 일반고객 분류
threshold = 27
user_max_product_count['user_type'] = user_max_product_count['product_count'].apply(lambda x: '사업자' if x >= threshold else '일반')

# 결과 출력
user_max_product_count



,user_id,product_count,user_type
0,1,9,일반
1,2,26,일반
2,3,11,일반
3,4,7,일반
4,5,12,일반
...,...,...,...
206204,206205,17,일반
206205,206206,24,일반
206206,206207,25,일반
206207,206208,28,사업자


In [27]:
#파이널 데이터 추출
duplicated_user_id_rows = df[df['user_id'].isin(user_max_product_count[user_max_product_count['user_type'] == '일반']['user_id'])]

#저장후 사용 하시면 됩니다.
duplicated_user_id_rows.to_csv('final_df2.csv', index=False)

In [28]:
# 전체 구매데이터 중 유기농 제품이 차지하는 비율이 32% 이상
## 유저별 유기농 제품 구매 비율 확인
user_organic_ratio = duplicated_user_id_rows.groupby('user_id')['is_organic'].mean()

## 32% 이상인 user_id 필터링
valid_organic_user_ids = user_organic_ratio[user_organic_ratio >= 0.32].index

## organic_customer 컬럼 생성
duplicated_user_id_rows['organic_customer'] = duplicated_user_id_rows['user_id'].isin(valid_organic_user_ids).astype(int)

duplicated_user_id_rows['organic_customer'].value_counts()

C:\Users\pc\AppData\Local\Temp\ipykernel_24248\3243746488.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicated_user_id_rows['organic_customer'] = duplicated_user_id_rows['user_id'].isin(valid_organic_user_ids).astype(int)


organic_customer
0    9937817
1    9049937
Name: count, dtype: int64

In [29]:
duplicated_user_id_rows.head()

,Unnamed: 0,user_id,order_id,product_id,product_name,aisle_id,aisle,department_id,department,order_number,reordered,is_organic,organic_customer
0,0,202279,2,33120,Organic Egg Whites,86,eggs,16,dairy eggs,3,1,True,1
1,1,202279,2,28985,Michigan Organic Kale,83,fresh vegetables,4,produce,3,1,True,1
2,2,202279,2,9327,Garlic Powder,104,spices seasonings,13,pantry,3,0,False,1
3,3,202279,2,45918,Coconut Butter,19,oils vinegars,13,pantry,3,1,False,1
4,4,202279,2,30035,Natural Sweetener,17,baking ingredients,13,pantry,3,0,False,1


In [24]:
duplicated_user_id_rows.to_csv('final_data.csv', index=False)